In [1]:
!pip install requests

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


I adapted [this notebook](https://github.com/Elucidation/Ngram-Tutorial/blob/master/NgramTutorial.ipynb) to Python 3. 

## IPython Notebook - N-gram Tutorial

Here is the explanation from the original author:

_What we want to do is build up a dictionary of N-grams, which are pairs, triplets or more (the N) of words that pop up in the training data, with the value being the number of times they showed up. After we have this dictionary, as a naive example we could actually predict sentences by just randomly choosing words within this dictionary and doing a weighted random sample of the connected words that are part of n-grams within the keys._

_Lets see how far we can get with N-grams without outside resources._

_We have a text file for [Pride and Prejudice from Project Gutenberg](https://www.gutenberg.org/ebooks/1342) stored as pg1342.txt in the same folder as our notebook, but also available online directly. Let's load the text to a string since it's only 701KB, which will fit in memory nowadays._

_**Note** : If we wanted to be more memory efficient we should parse the text file on a line or character by character basis, storing as needed, etc._

In [2]:
# Find the number links by looking on Project Gutenberg in the address bar for a book.
books = {'Pride and Prejudice': '1342',
         'Huckleberry Fin': '76',
         'Sherlock Holmes': '1661'}

book = books['Pride and Prejudice']

# Load text from Project Gutenberg URL
import requests
url_template = 'https://www.gutenberg.org/cache/epub/%s/pg%s.txt'

response = requests.get(url_template % (book, book), 'r')
txt = response.text

# See the number of characters and the first 50 characters to confirm it is there    
print(len(txt), ',', txt[:50] , '...')

762943 , ﻿The Project Gutenberg eBook of Pride and prejudic ...


Great, now lets split into words into a big list, splitting on anything non-alphanumeric [A-Za-z0-9] (as well as punctuation) and forcing everything lowercase:

In [3]:
import re
words = re.split('[^A-Za-z]+', txt.lower())
words = list(filter(None, words)) # Remove empty strings

# Print length of list
print(len(words))

131679


## N-grams generation
From this we can now generate N-grams, lets start with a 1-gram, basically the set of all the words

**Note** : One could use a dictionary instead of a set and keeping count of the occurances gives word frequency

In [4]:
# Create set of all unique words, this throws away any information about frequency however
gram1 = set(words)

print(len(gram1))

# Print 20 elements of the set only
print(list(gram1)[:20])

6981
['allayed', 'crisis', 'losses', 'finishing', 'polished', 'magnificent', 'suffered', 'letting', 'predict', 'follows', 'dependent', 'shortly', 'deal', 'unalterable', 'abusing', 'proficiency', 'deductible', 'moreover', 'judge', 'extracts']


Lets try and get the 2-gram now, which is pairs of words. Let's have a quick look to see the last 10 and how they look.

In [5]:
# See the last 10 pairs
for i in range(len(words)-10, len(words)-1):
    print(words[i], words[i+1])

subscribe to
to our
our email
email newsletter
newsletter to
to hear
hear about
about new
new ebooks


Okay, seems good, lets get all word pairs, and then generate a set of unique pairs from it

In [6]:
word_pairs = [(words[i], words[i+1]) for i in range(len(words)-1)]
print(len(word_pairs))

gram2 = set(word_pairs)
print(len(gram2))

# Print 20 elements from gram2
print(list(gram2)[:20])

131678
58228
[('offered', 'olive'), ('himself', 'he'), ('man', 'often'), ('a', 'quantity'), ('nor', 'falsely'), ('to', 'contradict'), ('more', 'gentle'), ('lately', 'gone'), ('never', 'allow'), ('am', 'sure'), ('himself', 'her'), ('hardly', 'help'), ('way', 'his'), ('her', 'nose'), ('subject', 'drop'), ('wishing', 'to'), ('illustration', 'they'), ('unjustly', 'she'), ('perceiving', 'whom'), ('forget', 'all')]


## N-Grams Frequency
Okay, that was fun, but this isn't enough, we need frequency if we want to have any sense of probabilities, which is what N-grams are about. Instead of using sets, lets create a dictionary with counts

In [7]:
gram1 = dict()

# Populate 1-gram dictionary
for word in words:
    if word in gram1:
        gram1[word] += 1
    else:
        gram1[word] = 1 # Start a new entry with 1 count since saw it for the first time.

# Turn into a list of (word, count) sorted by count from most to least
gram1 = sorted(gram1.items(), key=lambda item: -item[1])

# Print top 20 most frequent words
print([(word, freq) for word, freq in gram1[:20]])

[('the', 4853), ('to', 4407), ('of', 3964), ('and', 3838), ('her', 2284), ('i', 2122), ('a', 2096), ('in', 2054), ('was', 1878), ('she', 1751), ('that', 1661), ('it', 1605), ('not', 1528), ('you', 1451), ('he', 1361), ('his', 1303), ('be', 1281), ('as', 1240), ('had', 1186), ('with', 1150)]


For Pride and Prejudice, the words 'the', 'to', 'of', and 'and' were the top four most common words. Sounds about right, not too interesting yet, lets see what happens with 2-grams.

In [8]:
gram2 = dict()

# Populate 2-gram dictionary
for i in range(len(words)-1):
    key = (words[i], words[i+1])
    if key in gram2:
        gram2[key] += 1
    else:
        gram2[key] = 1

# Turn into a list of (word, count) sorted by count from most to least
gram2 = sorted(gram2.items(), key=lambda item: -item[1])

# Print top 20 most frequent words
print([(word, freq) for word, freq in gram2[:20]])

[(('of', 'the'), 542), (('to', 'be'), 448), (('in', 'the'), 440), (('i', 'am'), 312), (('to', 'the'), 281), (('mr', 'darcy'), 277), (('of', 'her'), 275), (('it', 'was'), 255), (('of', 'his'), 242), (('she', 'was'), 213), (('it', 'is'), 210), (('had', 'been'), 206), (('she', 'had'), 206), (('i', 'have'), 191), (('to', 'her'), 186), (('that', 'he'), 181), (('and', 'the'), 174), (('could', 'not'), 172), (('for', 'the'), 170), (('he', 'had'), 167)]


It looks like "of the" and "to be" are the top two most common 2-grams, sounds good.

## Next word prediction

What can we do with this? Well lets see what happens if we take a random word from all the words, and build a sentence by just choosing the most common pair that has that word as it's start.

In [9]:
start_word = words[int(len(words)/4)]
print(start_word)

him


I just went ahead and chose the word that appears $1/4$ of the way into words, random enough.

Now in a loop, iterate through the frequency list (most frequent first) and see if it matches the first word in a pair, if so, the next word is the second element in the word pair, and continue with that word. Stop after N words or the list does not contain that word.

**Note** : gram2 is a list that contains (key,value) where key is a word pair (first, second),
           so you need element[0][0] for first word and element [0][1] for second word

In [10]:
def get2GramSentence(word, n = 50):
    words = []
    for i in range(n):
        words.append(word)
        # Find Next word
        word = next((element[0][1] for element in gram2 if element[0][0] == word), None)
        if not word:
            break
    return ' '.join(words)

word = start_word
print("Start word:", word)

print('2-gram sentence: "', get2GramSentence(word, 20), '"')

Start word: him
2-gram sentence: " him to be so much as to be so much as to be so much as to be so much "


It gets stuck in a loop pretty much straight away. Not very interesting, try out other words and see what happens.

In [11]:
for word in ['and', 'he', 'she', 'when', 'john', 'never', 'i', 'how']:
    print("Start word:", word)
    print('2-gram sentence: "', get2GramSentence(word, 20), '"')

Start word: and
2-gram sentence: " and the same time to be so much as to be so much as to be so much as to "
Start word: he
2-gram sentence: " he had been so much as to be so much as to be so much as to be so much "
Start word: she
2-gram sentence: " she was not be so much as to be so much as to be so much as to be so "
Start word: when
2-gram sentence: " when she was not be so much as to be so much as to be so much as to be "
Start word: john
2-gram sentence: " john thorpe the same time to be so much as to be so much as to be so much as "
Start word: never
2-gram sentence: " never be so much as to be so much as to be so much as to be so much as "
Start word: i
2-gram sentence: " i am sure i am sure i am sure i am sure i am sure i am sure i am "
Start word: how
2-gram sentence: " how much as to be so much as to be so much as to be so much as to be "


## Weighted random choice based on frequency

**This is our simple probabilistic MLE N-gram model**

Same thing. Okay, lets randomly choose from the subset of all 2grams that matches the first word, using a weighted-probability based on counts.

In [12]:
import random
def weighted_choice(choices):
    total = sum(w for c, w in choices)
    r = random.uniform(0, total)
    upto = 0
    for c, w in choices:
        if upto + w > r:
            return c
        upto += w
    
def get2GramSentenceRandom(word, n = 50):
    words = []
    for i in range(n):
        words.append(word)
        # Get all possible elements ((first word, second word), frequency)
        choices = [element for element in gram2 if element[0][0] == word]
        if not choices:
            break
        
        # Choose a pair with weighted probability from the choice list
        word = weighted_choice(choices)[1]
    return ' '.join(words)

In [13]:
for word in ['and', 'he', 'she', 'when', 'john', 'never', 'i', 'how']:
    print("Start word:", word)
    print('2-gram sentence: "', get2GramSentenceRandom(word, 20), '"')

Start word: and
2-gram sentence: " and reluctant good wishes it between the neighbourhood it to my side and elizabeth of her purchases were alone than "
Start word: he
2-gram sentence: " he believed sincere pleasure of fancy themselves needlessly long outdone by that it without its propriety in her companions a "
Start word: she
2-gram sentence: " she was the door was not to be frightened by asking whether her has lydia s two youngest i propose "
Start word: when
2-gram sentence: " when i have a respect towards conversing easily with regret invectives against such an affection for fortune though it to "
Start word: john
2-gram sentence: " john with his entrance hall they were to dinner was expected for and various claims on to have talked incessantly "
Start word: never
2-gram sentence: " never wanted to ask of atonement he is dead i should suffer from a smile of affection of saying he "
Start word: i
2-gram sentence: " i will only the man as all means you must be exposing him more luc

Now that's way more interesting! Those are starting to look like sentences!

Let's try a longer sentence

In [14]:
word = 'it'
print("Start word:", word)
print('2-gram sentence: "', get2GramSentenceRandom(word, 50), '"')

Start word: it
2-gram sentence: " it give and blessing denied knowing it is the arts which she walked about the size of my sister elizabeth was of compliance for such spasms in a pleasant i that he will not one good opinion of preaching and had some others this effect of superior dancing at the "


Pretty cool, lets see what happens when we go to N-grams above 2.

## Tri-grams and more
Okay, let's create a Ngram generator that can let us make ngrams of arbitrary sizes

In [15]:
def generateNgram(n=1):
    gram = dict()
    
    # Some helpers to keep us crashing the PC for now
    assert n > 0 and n < 100
    
    # Populate N-gram dictionary
    for i in range(len(words)-(n-1)):
        key = tuple(words[i:i+n])
        if key in gram:
            gram[key] += 1
        else:
            gram[key] = 1

    # Turn into a list of (word, count) sorted by count from most to least
    gram = sorted(gram.items(), key=lambda item: -item[1])
    return gram

trigram = generateNgram(3)
# Print top 20 most frequent ngrams
print(trigram[:20])

[(('i', 'do', 'not'), 68), (('i', 'am', 'sure'), 64), (('project', 'gutenberg', 'tm'), 57), (('as', 'soon', 'as'), 56), (('she', 'could', 'not'), 53), (('that', 'he', 'had'), 37), (('in', 'the', 'world'), 36), (('copyright', 'by', 'george'), 35), (('by', 'george', 'allen'), 35), (('i', 'am', 'not'), 34), (('it', 'would', 'be'), 34), (('the', 'project', 'gutenberg'), 33), (('i', 'dare', 'say'), 30), (('it', 'was', 'not'), 30), (('that', 'he', 'was'), 30), (('mr', 'darcy', 's'), 30), (('as', 'well', 'as'), 29), (('could', 'not', 'be'), 29), (('would', 'have', 'been'), 28), (('that', 'it', 'was'), 28)]


In [16]:
def getNGramSentenceRandom(gram, word, n = 50):
    words = []
    for i in range(n):
        words.append(word)
        # Get all possible elements ((first word, second word), frequency)
        choices = [element for element in gram if element[0][0] == word]
        if not choices:
            break
        
        # Choose a pair with weighted probability from the choice list
        word = weighted_choice(choices)[1]
    return ' '.join(words)

for n in range(2,10):
    # Generate ngram list
    print(f"Generating {n}-gram list...")
    ngram = generateNgram(n)
    print("Done")
    
    # Try out a bunch of sentences
    for word in ['and', 'he', 'she', 'when', 'john', 'never', 'i', 'how']:
        print("Start word:", word)
        print(f'{n}-gram sentence: \n"{getNGramSentenceRandom(ngram, word, 20)}"')
        print()
        
    print('***************************************************************************')

Generating 2-gram list...
Done
Start word: and
2-gram sentence: 
"and mr bingley whose affection could she blushed and her sisters the short time to mr collins through the most"

Start word: he
2-gram sentence: 
"he replied darcy may wish his addressing him again when thus removed with only shook his mind was all about"

Start word: she
2-gram sentence: 
"she had been glad you but elizabeth for falling in spite of my feelings in any other way to her"

Start word: when
2-gram sentence: 
"when have no rest of their visitor was not forgot perhaps at the master soon after what she had not"

Start word: john
2-gram sentence: 
"john dashwood the evening between them but if your ladyship s regiment their aunt when the subject as they did"

Start word: never
2-gram sentence: 
"never saw much wretched suspense could not so daring to console lady catherine how ashamed of awkward as you to"

Start word: i
2-gram sentence: 
"i should communicate and prejudice far as possible the book mr collins s 

The sentences produced by higher-level N-gram looks almost like normal sentences if you squint a little!